In [1]:
from ghostnet.ghost_net import ghost_net
# from keras.datasets import mnist
import pandas as pd
import numpy as np
import torch
from torch.nn.utils.rnn import pack_padded_sequence
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
from torch import nn,optim
from torch.autograd import Variable
import torch.nn.functional as F
import matplotlib.pyplot as plt
import SimpleITK as sitk
import os

# 检验GPU是否可用
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

cuda:0


In [2]:
def dataLoad(path1, path2):
    paths1 = os.listdir(path1)
    paths2 = os.listdir(path2)
    dataset = []
    labelset = []
    
    for file in paths1:
        if file.find('mhd')>=0:
            filepath = os.path.join(path1, file)
            data = sitk.GetArrayFromImage(sitk.ReadImage(filepath))
            data = data.reshape((1,256,256))
            dataset.append(data)
            labelset.append(1)
            
    for file in paths2:
        if file.find('mhd')>=0:
            filepath = os.path.join(path2, file)
            data = sitk.GetArrayFromImage(sitk.ReadImage(filepath))
            data = data.reshape((1,256,256))
            dataset.append(data)
            labelset.append(0)
        
    dataset = np.array(dataset)
    labelset = np.array(labelset)
    return dataset, labelset

In [3]:
train_data, train_label = dataLoad('Training/Diseased', 'Training/Healthy')
test_data, test_label = dataLoad('Testing1/Diseased', 'Testing1/Healthy')

print(train_data.shape)
print(test_data.shape)


(378, 1, 256, 256)
(50, 1, 256, 256)


In [4]:
train_data = train_data.reshape((378,1,256,256))
test_data = test_data.reshape((-1,1,256,256))
train_data = torch.FloatTensor(train_data)
test_data = torch.FloatTensor(test_data)
train_label = torch.LongTensor(train_label)
test_label = torch.LongTensor(test_label)

In [5]:
test_label.shape

torch.Size([50])

In [6]:
class MyDataset(Dataset):
    
    # 初始化
    def __init__(self, pics, lable):
        # 读入数据
        self.data = pics
        self.lable = lable
        
    # 返回df的长度
    def __len__(self):
        return len(self.data)
    
    # 获取第idx+1列的数据
    def __getitem__(self, idx):
        return self.data[idx],self.lable[idx]

In [7]:
my_dataset_train = MyDataset(train_data, train_label)
my_dataset_test = MyDataset(test_data, test_label)
train_loader = DataLoader(dataset = my_dataset_train, batch_size = 8, shuffle = True)
test_loader = DataLoader(dataset = my_dataset_test, batch_size = 8, shuffle = True)

In [8]:
model = ghost_net(width_mult=1.0,num_classes=2).to(device)
# model = Net().to(device)
optimizer = optim.Adam(model.parameters(),lr=0.001)  # 优化方式为mini-batch momentum-SGD，并采用L2正则化（权重衰减）
criterion = nn.CrossEntropyLoss()

In [9]:
# model.train()
batch_idx = 1
epoches = 200
for epoch in range(epoches):
    batch_idx = 1
    model.train()
    for line in train_loader:
        data = line[0].to(device)
        label = line[1].to(device)
        output = model(data)
    #     print(output.size())
    #     print(label.size())
#         label = label.squeeze(1)
        output = torch.log_softmax(output,dim = 1)
        loss = criterion(output, label)
        optimizer.zero_grad() #这一步可太TM重要了
        loss.backward()
        optimizer.step()
#         print(loss)
        if(batch_idx)%10 == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.item()))
        batch_idx += 1
        
    correct = 0
    model.eval()
    for line in test_loader:
        data = line[0].to(device)
        label = line[1].to(device)
        output = model(data)
        output = torch.log_softmax(output,dim = 1)
        pred = output.max(1, keepdim=True)[1]# 找到概率最大的下标
        correct += pred.eq(label.view_as(pred)).sum().item()
    print("测试集上准确率为:",(correct/50)*100,"%")

Train Epoch: 0 [80/378 (21%)]	Loss: 0.378413
Train Epoch: 0 [160/378 (42%)]	Loss: 0.224237
Train Epoch: 0 [240/378 (62%)]	Loss: 0.360527
Train Epoch: 0 [320/378 (83%)]	Loss: 0.172558
测试集上准确率为: 76.0 %
Train Epoch: 1 [80/378 (21%)]	Loss: 0.327983
Train Epoch: 1 [160/378 (42%)]	Loss: 0.437496
Train Epoch: 1 [240/378 (62%)]	Loss: 0.385046
Train Epoch: 1 [320/378 (83%)]	Loss: 0.517461
测试集上准确率为: 54.0 %
Train Epoch: 2 [80/378 (21%)]	Loss: 0.426350
Train Epoch: 2 [160/378 (42%)]	Loss: 0.328404
Train Epoch: 2 [240/378 (62%)]	Loss: 0.074362
Train Epoch: 2 [320/378 (83%)]	Loss: 0.147389
测试集上准确率为: 32.0 %
Train Epoch: 3 [80/378 (21%)]	Loss: 0.934790
Train Epoch: 3 [160/378 (42%)]	Loss: 0.413333
Train Epoch: 3 [240/378 (62%)]	Loss: 0.309151
Train Epoch: 3 [320/378 (83%)]	Loss: 0.419823
测试集上准确率为: 74.0 %
Train Epoch: 4 [80/378 (21%)]	Loss: 0.214529
Train Epoch: 4 [160/378 (42%)]	Loss: 0.573270
Train Epoch: 4 [240/378 (62%)]	Loss: 0.592609
Train Epoch: 4 [320/378 (83%)]	Loss: 0.173599
测试集上准确率为: 66.0 %


KeyboardInterrupt: 